In [1]:
from entities import *
from constants import *
from simulations import *
from tournament_roster import *
from constants import TEAMS_DATA, STAT_EFFECTS

import pickle
import numpy as np
import pandas as pd
import openpyxl as xl

In [2]:
with open(TEAMS_DATA, 'rb') as f:
    teams = pickle.load(f)

with open(STAT_EFFECTS, 'rb') as f:
    stats = pickle.load(f)

In [3]:
wb = xl.load_workbook('resources/blank_bracket.xlsx', data_only=True)

tournament_roster = [teams[wb['Teams'][f'C{i}'].value] for i in range(2, 66)]
tournament_roster

[<Team: Abilene Christian Record: 7-3>,
 <Team: Air Force Record: 3-8>,
 <Team: Akron Record: 8-2>,
 <Team: Alabama Record: 7-2>,
 <Team: Alabama A&M Record: 4-4>,
 <Team: Alabama State Record: 3-6>,
 <Team: Albany (NY) Record: 3-7>,
 <Team: Alcorn State Record: 1-9>,
 <Team: American Record: 7-4>,
 <Team: Appalachian State Record: 5-5>,
 <Team: Arizona Record: 8-0>,
 <Team: Arizona State Record: 7-2>,
 <Team: Arkansas Record: 7-2>,
 <Team: Arkansas State Record: 7-3>,
 <Team: Arkansas-Pine Bluff Record: 1-9>,
 <Team: Army Record: 4-7>,
 <Team: Auburn Record: 7-3>,
 <Team: Austin Peay Record: 4-5>,
 <Team: Ball State Record: 3-6>,
 <Team: Baylor Record: 6-2>,
 <Team: Bellarmine Record: 4-5>,
 <Team: Belmont Record: 9-1>,
 <Team: Bethune-Cookman Record: 3-6>,
 <Team: Binghamton Record: 3-7>,
 <Team: Boise State Record: 6-3>,
 <Team: Boston College Record: 5-5>,
 <Team: Boston University Record: 4-6>,
 <Team: Bowling Green Record: 7-3>,
 <Team: Bradley Record: 6-4>,
 <Team: Brigham Young

In [4]:
def round_of_games(team_list: list, n: int=20, summary=False):
    winners, losers = [], []
    schools = []
    records = []
    analyses = []

    for i in range(0, len(team_list), 2):
        game = simulate_n_games(team_list[i], team_list[i+1], n, summary=summary)
        winners.append(teams[game['Winner']].name)
        losers.append(teams[game['Loser']].name)
        analyses.append(game['Analysis'])
        schools.append([game['Team 1'], game['Team 2']])
        records.append([game['Win Count 1'], game['Win Count 2']])
    
    return {'Winners': winners, 
            'Losers': losers, 
            'Analysis': analyses, 
            'Schools': schools,
            'Records': records
        }


def first_round(region: dict, n: int=20):
    first_round_teams = [teams[v] for v in region.values()]
    return round_of_games(first_round_teams, n=n)

In [5]:
midwest_first_round = first_round(midwest, n=50)
west_first_round    = first_round(west,    n=50)
south_first_round   = first_round(south,   n=50)
east_first_round    = first_round(east,    n=50)

Simulating Game: Arkansas-Pine Bluff vs. Army


100%|██████████| 50/50 [00:00<00:00, 119.44it/s]


Arkansas-Pine Bluff vs. Army Complete!


In [11]:
midwest_first_round

{'Winners': ['Chattanooga',
  'Clemson',
  'Coastal Carolina',
  'Colgate',
  'Colorado State',
  'Connecticut',
  'Cornell',
  'Creighton'],
 'Losers': ['Chicago State',
  'Cincinnati',
  'Cleveland State',
  'College of Charleston',
  'Colorado',
  'Columbia',
  'Coppin State',
  'Dartmouth'],
 'Analysis': ['', '', '', '', '', '', '', ''],
 'Schools': [[<Team: Chattanooga Record: 4-5>,
   <Team: Chicago State Record: 1-9>],
  [<Team: Cincinnati Record: 6-3>, <Team: Clemson Record: 10-3>],
  [<Team: Cleveland State Record: 3-8>, <Team: Coastal Carolina Record: 6-4>],
  [<Team: Colgate Record: 6-4>, <Team: College of Charleston Record: 4-5>],
  [<Team: Colorado Record: 8-1>, <Team: Colorado State Record: 7-2>],
  [<Team: Columbia Record: 9-1>, <Team: Connecticut Record: 8-1>],
  [<Team: Coppin State Record: 2-10>, <Team: Cornell Record: 5-5>],
  [<Team: Creighton Record: 5-4>, <Team: Dartmouth Record: 3-4>]],
 'Records': [[50, 0],
  [7, 43],
  [0, 50],
  [45, 5],
  [23, 27],
  [19, 31]

In [12]:
west_first_round

{'Winners': ['Auburn',
  'Baylor',
  'Belmont',
  'Bethune-Cookman',
  'Boise State',
  'Bowling Green',
  'Brigham Young',
  'Brown'],
 'Losers': ['Austin Peay',
  'Ball State',
  'Bellarmine',
  'Binghamton',
  'Boston College',
  'Boston University',
  'Bradley',
  'Bryant'],
 'Analysis': ['', '', '', '', '', '', '', ''],
 'Schools': [[<Team: Auburn Record: 7-3>, <Team: Austin Peay Record: 4-5>],
  [<Team: Ball State Record: 3-6>, <Team: Baylor Record: 6-2>],
  [<Team: Bellarmine Record: 4-5>, <Team: Belmont Record: 9-1>],
  [<Team: Bethune-Cookman Record: 3-6>, <Team: Binghamton Record: 3-7>],
  [<Team: Boise State Record: 6-3>, <Team: Boston College Record: 5-5>],
  [<Team: Boston University Record: 4-6>, <Team: Bowling Green Record: 7-3>],
  [<Team: Bradley Record: 6-4>, <Team: Brigham Young Record: 7-1>],
  [<Team: Brown Record: 5-6>, <Team: Bryant Record: 3-7>]],
 'Records': [[36, 14],
  [0, 50],
  [10, 40],
  [49, 1],
  [34, 16],
  [9, 41],
  [8, 42],
  [44, 6]]}

In [13]:
south_first_round

{'Winners': ['Buffalo',
  'Butler',
  'Cal State Fullerton',
  'California',
  'California Baptist',
  'Central Arkansas',
  'Central Connecticut State',
  'Charleston Southern'],
 'Losers': ['Bucknell',
  'Cal Poly',
  'Cal State Bakersfield',
  'Cal State Northridge',
  'Campbell',
  'Canisius',
  'Central Michigan',
  'Charlotte'],
 'Analysis': ['', '', '', '', '', '', '', ''],
 'Schools': [[<Team: Bucknell Record: 2-9>, <Team: Buffalo Record: 8-1>],
  [<Team: Butler Record: 7-2>, <Team: Cal Poly Record: 5-6>],
  [<Team: Cal State Bakersfield Record: 4-6>,
   <Team: Cal State Fullerton Record: 3-7>],
  [<Team: Cal State Northridge Record: 4-5>, <Team: California Record: 8-1>],
  [<Team: California Baptist Record: 7-3>, <Team: Campbell Record: 4-5>],
  [<Team: Canisius Record: 4-6>, <Team: Central Arkansas Record: 4-6>],
  [<Team: Central Connecticut State Record: 5-4>,
   <Team: Central Michigan Record: 3-6>],
  [<Team: Charleston Southern Record: 6-5>, <Team: Charlotte Record: 4-6>

In [14]:
east_first_round

{'Winners': ['Abilene Christian',
  'Akron',
  'Alabama State',
  'Albany (NY)',
  'Appalachian State',
  'Arizona',
  'Arkansas',
  'Army'],
 'Losers': ['Air Force',
  'Alabama',
  'Alabama A&M',
  'Alcorn State',
  'American',
  'Arizona State',
  'Arkansas State',
  'Arkansas-Pine Bluff'],
 'Analysis': ['', '', '', '', '', '', '', ''],
 'Schools': [[<Team: Abilene Christian Record: 7-3>,
   <Team: Air Force Record: 3-8>],
  [<Team: Akron Record: 8-2>, <Team: Alabama Record: 7-2>],
  [<Team: Alabama A&M Record: 4-4>, <Team: Alabama State Record: 3-6>],
  [<Team: Albany (NY) Record: 3-7>, <Team: Alcorn State Record: 1-9>],
  [<Team: American Record: 7-4>, <Team: Appalachian State Record: 5-5>],
  [<Team: Arizona Record: 8-0>, <Team: Arizona State Record: 7-2>],
  [<Team: Arkansas Record: 7-2>, <Team: Arkansas State Record: 7-3>],
  [<Team: Arkansas-Pine Bluff Record: 1-9>, <Team: Army Record: 4-7>]],
 'Records': [[46, 4],
  [34, 16],
  [1, 49],
  [47, 3],
  [8, 42],
  [46, 4],
  [50, 

In [16]:
# test
# round_of_games(midwest_first_round['Winners'])